In [1]:
import sys, os
import numpy as np
from scipy import stats
from iminuit import Minuit

sys.path.append('./python')
import analysis
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
###########################
# Setup Plotting Defaults #
###########################
# For more options see https://matplotlib.org/users/customizing.html

# Line styles
mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['lines.antialiased'] = True
mpl.rcParams['lines.dashed_pattern'] = 2.8, 1.5
mpl.rcParams['lines.dashdot_pattern'] = 4.8, 1.5, 0.8, 1.5
mpl.rcParams['lines.dotted_pattern'] = 1.1, 1.1
mpl.rcParams['lines.scale_dashes'] = True

# Default colors
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler('color',['cornflowerblue','forestgreen','maroon','goldenrod','firebrick','mediumorchid', 'navy', 'brown'])


# Fonts
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'CMU Serif'
mpl.rcParams['font.sans-serif'] = 'CMU Sans Serif, DejaVu Sans, Bitstream Vera Sans, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif'
mpl.rcParams['text.usetex'] = True

# Axes
mpl.rcParams['axes.linewidth'] = 1.0
mpl.rcParams['axes.labelsize'] = 24
mpl.rcParams['axes.labelpad'] = 9.0
                                                  
                                                  
# Tick marks - the essence of life
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['xtick.minor.width'] = 0.75
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.labelsize'] = 22
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.0
mpl.rcParams['ytick.minor.width'] = 0.75
mpl.rcParams['ytick.major.pad'] = 8
mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['ytick.minor.visible'] = True

# Legend
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['legend.frameon'] = True
mpl.rcParams['legend.framealpha'] = 1.
#mpl.rcParams['legend.edgecolor'] = 'black'
mpl.rcParams['legend.fancybox'] = True
mpl.rcParams['legend.borderpad'] = 0.4 # border whitespace
mpl.rcParams['legend.labelspacing'] = 0.5 # the vertical space between the legend entries
mpl.rcParams['legend.handlelength'] = 1.5 # the length of the legend lines
mpl.rcParams['legend.handleheight'] = 0.7 # the height of the legend handle
mpl.rcParams['legend.handletextpad'] = 0.5 # the space between the legend line and legend text
mpl.rcParams['legend.borderaxespad'] = 0.5 # the border between the axes and legend edge
mpl.rcParams['legend.columnspacing'] = 2.0 # column separation




# Figure size
mpl.rcParams['figure.figsize'] = 10, 6#, 12

# Save details
mpl.rcParams['savefig.bbox'] = 'tight'
mpl.rcParams['savefig.pad_inches'] = 0.1

In [3]:
c = 299792.458 # km/s

v0 = 220/c # unitless dispersion veloctiy 
vObs = 232/c # unitless boost velocity

In [4]:
ma = 2*np.pi*1e6
freqs = np.linspace(1e6-6, 1e6+10, 601)

template = analysis.getAxionTemplate(ma, freqs, v0, vObs)
data = stats.norm.rvs(loc = .1*template + 1, scale = .5, size = (1, len(template))) # needs to be at least (1, N)

In [5]:
fmin = ma / 2. / np.pi * (1-.5*(vObs + v0)**2)
fmax = ma / 2. / np.pi * (1+2*(vObs + v0)**2)

locs = np.where((freqs >= fmin)*(freqs <= fmax))[0]

#### Do the Analysis

In [6]:
# Format the freqs, data, and templates in convenient units
freq_center = (np.amax(freqs[locs]) + np.amin(freqs[locs])) / 2
freq_width = np.amax(freqs[locs]) - np.amin(freqs[locs])
freqs_no_dim = 2*(freqs-freq_center) / freq_width

data_rescale = np.mean(data)
data_no_dim = data / data_rescale

template_rescale = np.amax(template)
template_no_dim = template / template_rescale

#### Format the Optimizer Guess and Bounds

In [7]:
amp_guess = 0
amp_upper_bound = 2*(np.amax(data) - np.amin(data))
amp_lower_bound = -2*(np.amax(data) - np.amin(data))

mean_guess = np.mean(data_no_dim, axis = 1)
mean_upper_bounds = 2*mean_guess
mean_lower_bounds = np.zeros_like(mean_guess)

slope_guess = np.zeros_like(mean_guess)
slope_upper_bounds = .25*np.ones_like(slope_guess)
slope_lower_bounds = -.25*np.ones_like(slope_guess)

upper_bounds = np.append(amp_upper_bound, np.append(mean_upper_bounds, slope_upper_bounds))
lower_bounds = np.append(amp_lower_bound, np.append(mean_lower_bounds, slope_lower_bounds))
bounds = np.vstack((lower_bounds, upper_bounds)).T
guess = np.append(0, np.append(mean_guess, slope_guess))

#### Run the Optimization

In [8]:
loss = lambda x: analysis.NegLL(x, freqs_no_dim[locs], data_no_dim[:, locs], template_no_dim[locs])
grad = lambda x: analysis.NegLL_Jac(x, freqs_no_dim[locs], data_no_dim[:, locs], template_no_dim[locs])

m = Minuit(loss, guess, grad = grad)
m.errordef = 1
m.limits = bounds

# Initially fix all parameters
for i in range(len(guess)):
    m.fixed['x' + str(i)] = True
    
for i in range(data_no_dim.shape[0]):
    m.fixed['x' + str(i+1)] = False
    m.fixed['x' + str(i + 1 + data_no_dim.shape[0])] = False
    m.migrad()
    
    m.fixed['x' + str(i+1)] = True
    m.fixed['x' + str(i + 1 + data_no_dim.shape[0])] = True
    
Null_TS = m.fval
Null_Fit = np.array(m.values)

In [9]:
# Now release all parameters and fit the signal model hypothesis
m = Minuit(loss, Null_Fit, grad = grad)
m.errordef = 1
m.limits = bounds
m.migrad()

Signal_TS = m.fval
Signal_Fit = m.values

Discovery_TS = Null_TS - Signal_TS
Best_Fit = m.values['x0'] / template_rescale * data_rescale
Error = m.errors['x0'] / template_rescale * data_rescale

print(Discovery_TS, Best_Fit, Error)

0.4355524690503785 0.16701568497014077 0.2536074337594782


In [10]:
m.minos('x0', cl = .9)
limit =  Best_Fit + m.merrors['x0'].upper / template_rescale * data_rescale

# One Sigma Upper Errorbar for Limit
m.minos('x0', cl = stats.chi2.cdf((stats.chi2.ppf(.9, df = 1)**.5+1)**2, df = 1))
one_sigma_upper =  Best_Fit + m.merrors['x0'].upper / template_rescale * data_rescale

# Two Sigma Upper Errorbar for Limit
m.minos('x0', cl = stats.chi2.cdf((stats.chi2.ppf(.9, df = 1)**.5+2)**2, df = 1))
two_sigma_upper =  Best_Fit + m.merrors['x0'].upper / template_rescale * data_rescale

# One Sigma Lower Errorbar for Limit
m.minos('x0', cl = stats.chi2.cdf((stats.chi2.ppf(.9, df = 1)**.5-1)**2, df = 1))
one_sigma_lower =  Best_Fit + m.merrors['x0'].upper / template_rescale * data_rescale

# Now we'll use Minuit to construct a profiled likelihood
out = np.array(m.mnprofile('x0', size = 100, bound = 6, subtract_min = True))

# Finally, use Asimov to get the power-constrained limit threshold
pcl = (stats.chi2.ppf(.9, df = 1)**.5-1)*m.errors['x0']/ template_rescale * data_rescale
limit = np.maximum(limit, pcl) 

In [ ]:
mpl.rcParams['figure.figsize'] = 10, 5
plt.plot(out[0]/ template_rescale * data_rescale, out[1], lw = 2, color = 'black')
plt.axvline(Best_Fit, color = 'black', ls = ':', lw = 2, label = 'Best Fit')
plt.axvline(limit, color = 'red', lw = 3, label = '95\% Upper Limit')
plt.axvline(pcl, color = 'navy', ls = '--', lw = 2, label = 'PCL Threshold')
plt.axvspan(one_sigma_lower, one_sigma_upper, color = 'limegreen', alpha = .75)
plt.axvspan(one_sigma_upper, two_sigma_upper, color = 'gold', alpha = .75)

plt.ylim(0, 25)
plt.ylabel('Test Statistic')
plt.xlabel('Signal Amplitude')
plt.legend(fontsize = 16, ncol =3, loc = 'upper left')
plt.show()